<a href="https://colab.research.google.com/github/BiggusMaximus/Tensorflow_Certification_Preparation_Dicoding/blob/main/A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==2.7.0
!pip install pandas==1.3.4
!pip install PIL==8.4.0
!pip install tensorflow_datasets==4.4.0
!pip install numpy==1.21.4
!pip install scipy==1.7.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 665.5 MB 89.8 MB/s
     |████████████████████████████████| 1.3 MB 7.5 MB/s 
     |████████████████████████████████| 463 kB 61.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
    Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636
Looking in indexes: htt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 38.1 MB 386 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [3]:
import tensorflow as tf
import pandas as pd
import PIL
import tensorflow_datasets
import numpy as np
import scipy as sp
import platform

In [4]:
print(f"Python version : {platform.python_version()}")
print(f"tensorflow version : {tf.__version__}")
print(f"pandas version : {pd.__version__}")
print(f"pillow version : {PIL.__version__}")
print(f"tensorflow_datasets version : {tensorflow_datasets.__version__}")
print(f"numpy version : {np.version.version}")
print(f"scipy version : {sp.__version__}")

Python version : 3.7.13
tensorflow version : 2.7.0
pandas version : 1.3.4
pillow version : 7.1.2
tensorflow_datasets version : 4.4.0
numpy version : 1.21.6
scipy version : 1.7.3


In [5]:
# =======================================================================================
# PROBLEM A5
#
# Build and train a neural network model using the Sunspots.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from kaggle.com/robervalt/sunspots
#
# Desired MAE < 0.15 on the normalized dataset.
# ========================================================================================

import csv
import tensorflow as tf
import numpy as np
import urllib

# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_A5():
    data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/sunspots.csv'
    urllib.request.urlretrieve(data_url, 'sunspots.csv')

    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
      reader = csv.reader(csvfile, delimiter=',')
      next(reader)
      for row in reader:
        sunspots.append(float(row[2]))
        time_step.append(int(row[0]))

    time = np.array(time_step)
    series = np.array(sunspots)

    # Normalization Function. DO NOT CHANGE THIS CODE
    min=np.min(series)
    max=np.max(series)
    series -= min
    series /= max
    time=np.array(time_step)

    # DO NOT CHANGE THIS CODE
    split_time=3000
    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]

    # DO NOT CHANGE THIS CODE
    window_size=30
    batch_size=32
    shuffle_buffer_size=1000


    train_set=windowed_dataset(x_train, window_size=window_size,
                               batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)


    model=tf.keras.models.Sequential([
      tf.keras.layers.Dense(30, input_shape=[window_size], activation="relu"), 
      tf.keras.layers.Dense(10, activation="relu"),
      tf.keras.layers.Dense(1)
    ])

    # YOUR CODE
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
    optimizer = tf.keras.optimizers.SGD(momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
    model.fit(train_set, epochs=100, callbacks=[lr_schedule])
    return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_A5()
    model.save("model_A5.h5")

Epoch 1/100
93/93 [==============================] - 1s 8ms/step - loss: 0.0721 - mae: 0.3014 - lr: 1.0000e-08
Epoch 2/100
93/93 [==============================] - 1s 4ms/step - loss: 0.0721 - mae: 0.3014 - lr: 1.1220e-08
Epoch 3/100
93/93 [==============================] - 1s 5ms/step - loss: 0.0721 - mae: 0.3014 - lr: 1.2589e-08
Epoch 4/100
93/93 [==============================] - 1s 4ms/step - loss: 0.0721 - mae: 0.3014 - lr: 1.4125e-08
Epoch 5/100
93/93 [==============================] - 1s 5ms/step - loss: 0.0721 - mae: 0.3014 - lr: 1.5849e-08
Epoch 6/100
93/93 [==============================] - 0s 4ms/step - loss: 0.0721 - mae: 0.3013 - lr: 1.7783e-08
Epoch 7/100
93/93 [==============================] - 1s 4ms/step - loss: 0.0721 - mae: 0.3013 - lr: 1.9953e-08
Epoch 8/100
93/93 [==============================] - 1s 4ms/step - loss: 0.0721 - mae: 0.3013 - lr: 2.2387e-08
Epoch 9/100
93/93 [==============================] - 1s 4ms/step - loss: 0.0721 - mae: 0.3013 - lr: 2.5119e-08
E